In [1]:
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np 
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GroupShuffleSplit
import joblib
from copy import deepcopy
import dask.dataframe as dd

import gc
import random
import pickle
from scipy.stats import pearsonr

# torch.manual_seed(42)
# np.random.seed(42)
# random.seed(42)


In [2]:
cell_type_list = ['CD4 T', 'CD8 T', 'B', 'Macro']
feature_extractor = "PC-CHiP"
output_name = "CD48_pchip_no_test"

In [3]:
#PC-CHiP
features_dir = "/home/evos/Outputs/CRC/FF/1_extract_histopathological_features/features.txt"
features_histo = pd.read_csv(features_dir, sep="\t")

#UNI
# features_dir = "/home/evos/Outputs/CRC/UNI_features_TCGA/FFPE"
# features_histo_dd = dd.read_parquet(f"{features_dir}/FFPE_features_batch_*.parquet")
# features_histo = features_histo_dd.compute()
# print(len(features_histo)) 

In [4]:
features_orion_dir = "/home/evos/Outputs/CRC/Orion/1_extract_histopathological_features_macenko/features.txt"
features_orion = pd.read_csv(features_orion_dir, sep="\t")
features_orion['slide_submitter_id'] = features_orion['tile_ID'].str.rsplit("-registered", n=1).str[0] + "-registered"
print(len(features_orion['slide_submitter_id'].unique()))

41


In [5]:
# def add_labels(features_df, rectangle_estimation, cell_type_list):
#     scaling = False 

#     if scaling == True:
#         for cell_type in cell_type_list:
#             minmax_scaler = MinMaxScaler()
#             column = rectangle_estimation[cell_type].values.reshape(-1, 1)
#             rectangle_estimation[cell_type] = minmax_scaler.fit_transform(column).flatten()

#     def get_base_id(sample_id):
#         return sample_id[:-1]  # Removes the last character

#     for cell_type in cell_type_list:
#         features_df[cell_type] = features_df['sample_submitter_id'].apply(get_base_id).map(rectangle_estimation[cell_type])
    
#     return features_df

# output_path = f"/home/evos/Outputs/CRC/deconvolution_sc/final_sc_deconv_results/{ann}"
# rectangle_estimation_ =  pd.read_csv(f"{output_path}/cell_frac_{ann}.csv", sep=",", index_col=0)
# rectangle_estimation = rectangle_estimation_.where(rectangle_estimation_ >= 0, 0)
# print(rectangle_estimation.columns)

# features_hist = add_labels(features_histo, rectangle_estimation, cell_type_list)

In [6]:
def add_labels(features_df, rectangle_estimation_dict, cell_type_list):
    scaling = False  

    if scaling:
        for cell_type, data in rectangle_estimation_dict.items():
            minmax_scaler = MinMaxScaler()
            column = data[cell_type].values.reshape(-1, 1)
            rectangle_estimation_dict[cell_type][cell_type] = minmax_scaler.fit_transform(column).flatten()

    def get_base_id(sample_id):
        return sample_id[:-1]  # Removes the last character

    for cell_type in cell_type_list:
        if cell_type in rectangle_estimation_dict:
            features_df[cell_type] = (
                features_df["sample_submitter_id"]
                .apply(get_base_id)
                .map(rectangle_estimation_dict[cell_type][cell_type])
            )
    
    return features_df


# Define paths
output_path = "/home/evos/Outputs/CRC/deconvolution_sc/final_sc_deconv_results"
# Load the different versions
rectangle_estimation_ann1 = pd.read_csv(f"{output_path}/ann1/cell_frac_ann1.csv", sep=",", index_col=0).where(lambda x: x >= 0, 0)
rectangle_estimation_ann2 = pd.read_csv(f"{output_path}/ann2/cell_frac_ann2.csv", sep=",", index_col=0).where(lambda x: x >= 0, 0)

# Define which cell type uses which version
cell_type_to_ann = {
    "B": rectangle_estimation_ann1,
    "T": rectangle_estimation_ann1,
    "EpiT": rectangle_estimation_ann1,  
    "CD4 T": rectangle_estimation_ann2,
    "CD8 T": rectangle_estimation_ann2,  
    "Macro": rectangle_estimation_ann1,
    "Stromal": rectangle_estimation_ann1,
    "Endothelial": rectangle_estimation_ann2,
}

# Apply function
features_hist = add_labels(features_histo, cell_type_to_ann, cell_type_list)
print(features_hist.columns)
del features_histo

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       'tile_ID', 'sample_submitter_id', 'slide_submitter_id', 'Section',
       'Coord_X', 'Coord_Y', 'CD4 T', 'CD8 T', 'B', 'Macro'],
      dtype='object', length=1547)


In [7]:
def patch_sampling(features_hist, n_patches_per_slide, min_patches_per_slide=50, sample_all=False):
    """
    Samples a fixed number of patches from each slide in the features_hist DataFrame.
    Slides with fewer than min_patches_per_slide patches are excluded.
    
    Parameters:
    - features_hist: pandas DataFrame containing the feature data for each slide
    - n_patches_per_slide: Number of patches to sample per slide (ignored if sample_all=True)
    - min_patches_per_slide: Minimum number of patches required to include a slide in the sampled data
    - sample_all: If True, all patches are sampled, otherwise a fixed number is sampled per slide
    
    Returns:
    - features_hist_sampled: A new DataFrame with sampled patches for each slide
    """
    features_hist_sampled = []

    # Group by 'slide_submitter_id' to sample patches for each slide
    for slide_id, group in features_hist.groupby('slide_submitter_id'):
        # Exclude slides with fewer than the minimum number of patches
        if len(group) < min_patches_per_slide:
            continue

        if sample_all:
            # If sampling all patches, just use the entire group
            sampled_group = group
        else:
            # Otherwise, sample a fixed number of patches
            n_patches = min(n_patches_per_slide, len(group))
            sampled_group = group.sample(n=n_patches, random_state=42)

        features_hist_sampled.append(sampled_group)

    # Concatenate all the sampled patches back into a single DataFrame
    features_hist_sampled = pd.concat(features_hist_sampled, axis=0)

    return features_hist_sampled


In [8]:
# Initialize GroupShuffleSplit
df_train_val = features_hist.copy()
del features_hist

#Apply sampling on df_train_val dataset
df_train_val = patch_sampling(df_train_val,300,50,False)

# Verify the split
print(len(df_train_val['slide_submitter_id'].unique()))

kf = KFold(n_splits=5, shuffle=True, random_state=40)  # 5 folds with shuffling
folds = []

print(len)
unique_slide_ids = df_train_val['slide_submitter_id'].unique()
print(len(unique_slide_ids))

# Generate indices for cross-validation
for train_idx, val_idx in kf.split(unique_slide_ids):
    train_slide_ids = unique_slide_ids[train_idx]
    val_slide_ids = unique_slide_ids[val_idx]
    folds.append((train_slide_ids, val_slide_ids))

for fold, (train_ids, val_ids) in enumerate(folds):
    print(f"Fold {fold + 1}:")
    print(f"  Training slides: {len(train_ids)}")
    print(f"  Validation slides: {len(val_ids)}")

220573


221
54
<built-in function len>
221
Fold 1:
  Training slides: 176
  Validation slides: 45
Fold 2:
  Training slides: 177
  Validation slides: 44
Fold 3:
  Training slides: 177
  Validation slides: 44
Fold 4:
  Training slides: 177
  Validation slides: 44
Fold 5:
  Training slides: 177
  Validation slides: 44
Test slides: 54


In [9]:
class MILDataset(Dataset):
    def __init__(self, bag_data, label_columns):
        """
        Initialize the dataset.
        
        Args:
            bag_data (dict): A dictionary with slide ids as keys, containing feature arrays and corresponding labels.
            label_columns (list): List of column names that represent the cell type labels.
        """
        self.bag_data = bag_data
        self.slide_ids = list(bag_data.keys())
        self.label_columns = label_columns # Store the label columns

    def __len__(self):
        return len(self.bag_data)

    def __getitem__(self, idx):
        slide_id = self.slide_ids[idx]
        features = torch.tensor(self.bag_data[slide_id]['features'], dtype=torch.float32)
        # Get the labels for the multiple cell types
        labels = torch.tensor([self.bag_data[slide_id]['label'][col] for col in self.label_columns], dtype=torch.float32)
        
        return features, labels

def collate_fn(batch):
    features = [item[0] for item in batch]  # List of feature tensors
    labels = torch.stack([item[1] for item in batch])  # Stack the labels into a single tensor (shape: [batch_size, num_cell_types])
    return features, labels


class MultiHeadMILModel(nn.Module):
    def __init__(self, input_size, num_cell_types):
        super(MultiHeadMILModel, self).__init__()
        self.shared_layer = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            #nn.Dropout(0.2),
        )
        self.heads = nn.ModuleList([nn.Linear(256, 1) for _ in range(num_cell_types)])

    def forward(self, x):
        shared_features = self.shared_layer(x)
        outputs = [torch.sigmoid(head(shared_features)) for head in self.heads]
        return torch.cat(outputs, dim=1)  # Shape: [batch_size, num_cell_types]


def pearson_correlation_loss(pred, target):
    pred = pred - pred.mean(dim=0)
    target = target - target.mean(dim=0)
    
    numerator = torch.sum(pred * target, dim=0)
    denominator = torch.sqrt(torch.sum(pred ** 2, dim=0) * torch.sum(target ** 2, dim=0))
    
    return -torch.mean(numerator / (denominator + 1e-8))  # Add small epsilon for stability

In [10]:
def standardize_features_incremental(df_train, df_val, feature_columns=None):
    """
    Standardizes the features in df_train and df_val using an incremental approach.
    This avoids fitting the scaler to the entire dataset at once, making it more memory efficient.
    
    Parameters:
    - df_train: pandas DataFrame containing the training data
    - df_val: pandas DataFrame containing the validation data
    - feature_columns: list of feature column names (if None, assumes all columns except 'label' and identifiers)
    
    Returns:
    - df_train: pandas DataFrame with standardized features for training
    - df_val: pandas DataFrame with standardized features for validation
    - scaler: StandardScaler object fitted on the training data
    """
    if feature_columns is None:
        feature_columns = df_train.columns[:-2]  # Assuming last two columns are 'label' and identifier
    
    # Initialize variables for incremental mean and variance computation
    n_samples = 0
    mean_sum = np.zeros(len(feature_columns))
    var_sum = np.zeros(len(feature_columns))
    
    # Incrementally compute the mean and variance for the training data
    for slide_id, group in df_train.groupby('slide_submitter_id'):
        features = group[feature_columns].values
        n_samples += features.shape[0]
        mean_sum += features.sum(axis=0)
        var_sum += (features ** 2).sum(axis=0)
    
    # Final mean and variance
    mean = mean_sum / n_samples
    variance = (var_sum / n_samples) - (mean ** 2)
    std_dev = np.sqrt(np.maximum(variance, 1e-8))  # Avoid division by zero
    
    # Apply the standardization to both the training and validation data
    df_train[feature_columns] = (df_train[feature_columns] - mean) / std_dev
    df_val[feature_columns] = (df_val[feature_columns] - mean) / std_dev
    
    # Create a scaler object that contains the mean and standard deviation
    scaler = StandardScaler()
    scaler.mean_ = mean
    scaler.scale_ = std_dev
    scaler.var_ = variance
    
    return df_train, df_val, scaler


In [11]:
class MultiLabelMSELoss(nn.Module):
    def __init__(self):
        super(MultiLabelMSELoss, self).__init__()

    def forward(self, predictions, labels):
        # Assuming predictions is [batch_size, num_labels] and labels is also [batch_size, num_labels]
        # Compute MSE for each label independently
        loss_per_label = torch.mean((predictions - labels) ** 2, dim=0)  # MSE for each label
        total_loss = torch.mean(loss_per_label)  # Average across all labels

        return total_loss

class MultiLabelPearsonLoss(nn.Module):
    def __init__(self):
        super(MultiLabelPearsonLoss, self).__init__()

    def forward(self, predictions, labels):
        """
        Compute multi-label Pearson correlation loss.
        predictions: [batch_size, num_labels]
        labels: [batch_size, num_labels]
        """
        eps = 1e-6  # Small value to avoid division by zero

        # Compute means
        pred_mean = predictions.mean(dim=0, keepdim=True)
        label_mean = labels.mean(dim=0, keepdim=True)

        # Compute numerator (covariance)
        covariance = ((predictions - pred_mean) * (labels - label_mean)).sum(dim=0)

        # Compute denominator (product of standard deviations)
        pred_std = torch.sqrt(((predictions - pred_mean) ** 2).sum(dim=0) + eps)
        label_std = torch.sqrt(((labels - label_mean) ** 2).sum(dim=0) + eps)

        # Pearson correlation per label
        pearson_corr = covariance / (pred_std * label_std + eps)

        # Loss: maximize correlation → minimize (1 - correlation)
        loss = -pearson_corr.mean()

        return loss

In [12]:
# Initialize model
criterion = MultiLabelPearsonLoss()
#criterion = pearson_correlation_loss
num_epochs = 20
n_cell_type = len(cell_type_list)
print(n_cell_type)
if feature_extractor == "PC-CHiP":
    input_size=1536
elif feature_extractor == "UNI":
    input_size=1024


for fold, (train_ids, val_ids) in enumerate(folds):
    model = MultiHeadMILModel(input_size=input_size, num_cell_types=n_cell_type)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    #optimizer = optim.Adam(model.parameters(), lr=0.001)
    print(f"Training Fold {fold + 1}")
    
    features_ = df_train_val.copy()
    # Create training and validation datasets
    df_train = features_[features_['slide_submitter_id'].isin(train_ids)]
    df_val = features_[features_['slide_submitter_id'].isin(val_ids)]

    # Count the number of tiles (patches) in training and validation sets
    train_tiles_count = df_train['tile_ID'].nunique()  # or use .shape[0] for number of rows
    val_tiles_count = df_val['tile_ID'].nunique()      # assuming 'tile_ID' is the identifier for each patch
    
    print(f"Number of tiles in training set for Fold {fold + 1}: {train_tiles_count}")
    print(f"Number of tiles in validation set for Fold {fold + 1}: {val_tiles_count}")
    
    remove_columns = ['Unnamed: 0', 'tile_ID', 'sample_submitter_id', 'Section', 'Coord_X', 'Coord_Y']
    # Clean the datasets
    df_train_clean = df_train.drop(columns=remove_columns)
    print(df_train_clean.columns)
    df_val_clean = df_val.drop(columns=remove_columns)

    #df_train_clean, df_val_clean, scaler = standardize_features_incremental(df_train_clean, df_val_clean)

    #Standardize features (fit on training data and transform both train and validation data)
    feature_columns = df_train_clean.columns[:(-1-n_cell_type)]  # Select all feature columns (so no label columns and no 'slide_submitter_id')
    scaler = StandardScaler()
    df_train_clean[feature_columns] = scaler.fit_transform(df_train_clean[feature_columns])
    df_val_clean[feature_columns] = scaler.transform(df_val_clean[feature_columns])
    
    bag_data_train = {}
    for slide_id, group in df_train_clean.groupby('slide_submitter_id'):
        features = group.iloc[:, 0:(-1-n_cell_type)].values
        labels = group[cell_type_list].iloc[0].to_dict()  # Adjust this part to match your label columns
        bag_data_train[slide_id] = {'features': features, 'label': labels}

    # For validation data
    bag_data_val = {}
    for slide_id, group in df_val_clean.groupby('slide_submitter_id'):
        features = group.iloc[:, 0:(-1-n_cell_type)].values
        labels = group[cell_type_list].iloc[0].to_dict()
        bag_data_val[slide_id] = {'features': features, 'label': labels}

    # Create datasets and loaders
    mil_dataset_train = MILDataset(bag_data_train, cell_type_list)
    mil_loader_train = DataLoader(mil_dataset_train, batch_size=28, collate_fn=collate_fn, shuffle=True)

    mil_dataset_val = MILDataset(bag_data_val, cell_type_list)
    mil_loader_val = DataLoader(mil_dataset_val, batch_size=28, collate_fn=collate_fn, shuffle=False)

    #TRAIN THE MODEL
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        total_loss = 0.0

        for batch_features, batch_labels in mil_loader_train:
            # Shape of batch_features: [batch_size, num_patches, num_features]
            # Shape of batch_labels: [batch_size, num_labels]
            optimizer.zero_grad()  # Zero gradients
            batch_predictions = []
                
            for features in batch_features:  # Iterate through each tensor in the batch
            # Shape of 'features': [num_patches, num_features]
                patch_predictions = model(features)  #Shape of patch_predictions: [num_patches, num_labels]
                aggregated_prediction = patch_predictions.mean(dim=0)  # Aggregate predictions
                batch_predictions.append(aggregated_prediction)

            batch_predictions = torch.stack(batch_predictions)

            #batch_labels = batch_labels.unsqueeze(1)
            loss = criterion(batch_predictions, batch_labels)
            total_loss += loss.item()
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
        
        # Validate the model after each epoch
        model.eval()  # Set model to evaluation mode
        total_val_loss = 0.0

        with torch.no_grad():  # No gradient calculation needed during validation
            for val_features, val_labels in mil_loader_val:
                batch_predictions = []

                for features in val_features:  # Iterate through each tensor in the batch
                    patch_predictions = model(features)  # Get predictions for the patches
                    aggregated_prediction = patch_predictions.mean(dim=0)  # Aggregate predictions
                    batch_predictions.append(aggregated_prediction)

                batch_predictions = torch.stack(batch_predictions)
        
                #val_labels = val_labels.unsqueeze(1)
                val_loss = criterion(batch_predictions, val_labels)
                total_val_loss += val_loss.item()
        
        avg_train_loss = total_loss / len(mil_loader_train)
        avg_val_loss = total_val_loss / len(mil_loader_val)
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")
        
        #torch.cuda.empty_cache()
        #gc.collect()
        
    torch.save(model.state_dict(), f"saved_models/model_fold_{fold}.pth")
    joblib.dump(scaler, f"saved_scalers/scaler_fold_{fold}.pkl")    


4
Training Fold 1
Number of tiles in training set for Fold 1: 48567
Number of tiles in validation set for Fold 1: 12817
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '1531', '1532', '1533', '1534', '1535', 'slide_submitter_id', 'CD4 T',
       'CD8 T', 'B', 'Macro'],
      dtype='object', length=1541)
Epoch 1/20, Training Loss: -0.2017259650996753, Validation Loss: -0.0119333416223526
Epoch 2/20, Training Loss: -0.4865839055606297, Validation Loss: -0.15196972456760705
Epoch 3/20, Training Loss: -0.49112526008061, Validation Loss: -0.20200876519083977
Epoch 4/20, Training Loss: -0.5563306297574725, Validation Loss: -0.21154868975281715
Epoch 5/20, Training Loss: -0.5783770254680088, Validation Loss: -0.19928539171814919
Epoch 6/20, Training Loss: -0.5874112801892417, Validation Loss: -0.21351158246397972
Epoch 7/20, Training Loss: -0.6579114624432155, Validation Loss: -0.2337368205189705
Epoch 8/20, Training Loss: -0.673881539276668, Validation Loss: -0.275

In [13]:
class MILDatasetTestOrion(Dataset):
    def __init__(self, bag_data):
        self.bag_data = bag_data
        self.slide_ids = list(bag_data.keys())

    def __len__(self):
        return len(self.bag_data)

    def __getitem__(self, idx):
        slide_id = self.slide_ids[idx]
        features = torch.tensor(self.bag_data[slide_id]['features'], dtype=torch.float32)
        patch_ids = self.bag_data[slide_id]['patch_ids']  # Patch IDs for the slide
        return features, patch_ids  # Return features and patch IDs for prediction

remove_columns_test = ['Unnamed: 0', 'sample_submitter_id', 'Section', 'Coord_X', 'Coord_Y']
df_test_orion = features_orion.drop(columns=remove_columns_test)

print(len(df_test_orion.columns))

bag_data_test_orion = {}
for slide_id, group in df_test_orion.groupby('slide_submitter_id'):
    features = group.iloc[:, 0:(-2)].values  # All feature columns, adjust as needed
    patch_ids = group['tile_ID'].tolist()  # Add patch IDs for each slide
    bag_data_test_orion[slide_id] = {'features': features, 'patch_ids': patch_ids}

1542
1538


In [16]:
#models = torch.load("models_all_folds.pth", weights_only=True)
# Initialize dictionaries for models and scalers
models = {}
scalers = {}

# Load models and scalers for each fold
for fold in range(5):  # Assuming 5 folds
    # Load model
    model = MultiHeadMILModel(input_size=input_size, num_cell_types=n_cell_type)# Instantiate the model architecture
    model.load_state_dict(torch.load(f"saved_models/model_fold_{fold}.pth"))
    model.eval()  # Set the model to evaluation mode
    models[fold] = model  # Add to the dictionary

    # Load scaler
    scaler = joblib.load(f"saved_scalers/scaler_fold_{fold}.pkl")
    scalers[fold] = scaler  # Add to the dictionary


/tmp/ipykernel_59335/3088891258.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"saved_models/model_fold_{fold}.pth"))


In [20]:
# Initialize containers for cross-fold results
all_final_predictions = {cell_type: [] for cell_type in cell_type_list}  # For storing final predictions per cell type
all_patch_probabilities = {cell_type: [] for cell_type in cell_type_list}  # For storing patch probabilities per cell type

all_patch_IDs_orion = []
# Populate the list with patch IDs from all slides
for slide_id, group in bag_data_test_orion.items():
    all_patch_IDs_orion.extend(group['patch_ids'])

for fold in range(5):  # Assuming 5 folds
    print(f"Evaluating Fold {fold + 1}")
    
    # Retrieve the model and scaler for the current fold
    model = models[fold]
    scaler = scalers[fold]
    
    # Normalize the test data using the current fold's scaler
    normalized_bag_data_orion = deepcopy(bag_data_test_orion)
    for slide_id in normalized_bag_data_orion:
        # Create a DataFrame with the same column names used during training
        features_df = pd.DataFrame(
            normalized_bag_data_orion[slide_id]['features'], 
            columns=scaler.feature_names_in_  # Use feature names from the scaler
        )
        # Normalize using the scaler
        normalized_bag_data_orion[slide_id]['features'] = scaler.transform(features_df)
    
    # Create the test dataset and DataLoader
    mil_dataset_orion = MILDatasetTestOrion(normalized_bag_data_orion)
    mil_loader_orion = DataLoader(mil_dataset_orion, batch_size=1, shuffle=False)
    
    # Collect predictions for the current fold
    fold_patch_probabilities = {cell_type: [] for cell_type in cell_type_list}  # For storing patch-level predictions per cell type
    fold_final_predictions = {cell_type: [] for cell_type in cell_type_list}  # For storing final aggregated predictions per cell type
    
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for features, patch_ids in mil_loader_orion:
            # Get patch probabilities (since each bag has size 1)
            patch_predictions = model(features.squeeze(0))  # Remove extra dimension
            
            # Collect patch probabilities for each cell type
            for idx, cell_type in enumerate(cell_type_list):
                fold_patch_probabilities[cell_type].append(patch_predictions[:,idx].numpy())  # Store the prediction for the specific cell type

            # Aggregate to get the final prediction (mean of patch probabilities) for each cell type
            for cell_type in cell_type_list:
                final_prediction = patch_predictions[idx].mean().item()
                fold_final_predictions[cell_type].append(final_prediction)
    
    # Store fold results for each cell type
    for cell_type in cell_type_list:
        all_patch_probabilities[cell_type].append(fold_patch_probabilities[cell_type])
        all_final_predictions[cell_type].append(fold_final_predictions[cell_type])


Evaluating Fold 1
Evaluating Fold 2
Evaluating Fold 3
Evaluating Fold 4
Evaluating Fold 5


In [21]:
print(len(all_patch_IDs_orion))
average_patch_probabilities_orion = {}

for cell_type in cell_type_list:
    print(f"Results for {cell_type} cell type:")

    # Calculate average final predictions for this cell type
    average_final_predictions = np.mean(all_final_predictions[cell_type], axis=0)

    # Concatenate patch probabilities for this cell type
    concatenated_patch_probabilities = []
    for fold in range(5):
        concatenated_patches = np.concatenate(all_patch_probabilities[cell_type][fold], axis=0)
        concatenated_patch_probabilities.append(concatenated_patches)
    
    # Average the concatenated patch probabilities for the current cell type
    average_patch_probabilities = np.mean(concatenated_patch_probabilities, axis=0)
    # Add the result to the dictionary
    average_patch_probabilities_orion[cell_type] = average_patch_probabilities

    # Print average final predictions and true labels
    print("Average final predictions:", average_final_predictions)

192690
Results for CD4 T cell type:
Average final predictions: [0.292956   0.48908759 0.15840869 0.17797678 0.10452881 0.20074659
 0.21284911 0.43890408 0.29662629 0.29043722 0.17358346 0.38925174
 0.07060091 0.06831405 0.3981499  0.18360193 0.411857   0.36778786
 0.28382655 0.10866483 0.35030594 0.24087923 0.185508   0.4065733
 0.10769923 0.17673828 0.27067699 0.01551191 0.05228679 0.25357209
 0.23071566 0.45553837 0.30896691 0.41917271 0.50811113 0.49043934
 0.23614226 0.51505275 0.40114164 0.37701266 0.21340441]
Results for CD8 T cell type:
Average final predictions: [0.292956   0.48908759 0.15840869 0.17797678 0.10452881 0.20074659
 0.21284911 0.43890408 0.29662629 0.29043722 0.17358346 0.38925174
 0.07060091 0.06831405 0.3981499  0.18360193 0.411857   0.36778786
 0.28382655 0.10866483 0.35030594 0.24087923 0.185508   0.4065733
 0.10769923 0.17673828 0.27067699 0.01551191 0.05228679 0.25357209
 0.23071566 0.45553837 0.30896691 0.41917271 0.50811113 0.49043934
 0.23614226 0.51505275

In [22]:
# Initialize the final DataFrame list
dataframes = []

# Iterate through the patch IDs (assumed to be ordered in the same way across cell types)
for idx, patch_ids in enumerate(all_patch_IDs_orion):  # Iterate through the list of patch IDs
    row_data = {'tile_ID': patch_ids}  # Start the row with the patch IDs
    # Add the corresponding probabilities for each cell type
    for cell_type in cell_type_list:
        row_data[cell_type] = average_patch_probabilities_orion[cell_type][idx]

    df = pd.DataFrame([row_data])
    dataframes.append(df)

# Concatenate all DataFrames into one big DataFrame and add columns
final_dataframe = pd.concat(dataframes, ignore_index=True)
final_dataframe['tile_ID'] = final_dataframe['tile_ID'].astype(str)
split_columns = final_dataframe['tile_ID'].str.rsplit("_", n=2, expand=True)
final_dataframe['slide_id'] = split_columns[0]
final_dataframe['Coord_X'] = split_columns[1]
final_dataframe['Coord_Y'] = split_columns[2]

# Set the output directory
full_output_dir = "/home/evos/Outputs/CRC/Orion/MIL_results/MIL_multiple_cell_types"

# Save the final DataFrame to a CSV file
final_dataframe.to_csv(f"{full_output_dir}/{output_name}.csv", sep="\t", index=False)
